In [30]:
import pandas as pd
import numpy as np
import datetime,datetime as dtt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import CheckboxGroup

In [5]:
def infect(source, *arg):
    '''
    Prepare data from original. Add Wold count, null selection, daily increase
    Returns ColumnsDataSource
    '''
    df = pd.read_csv(s0)
    df.drop(['Lat','Long'], axis = 1, inplace=True)
    # add World count
    df = df.append( pd.Series([np.NAN, 'World'], index = ['Province/State','Country/Region']).append(df.sum()), ignore_index =True)
    #add Null selection
    df = df.append(pd.Series([np.NAN, 'No selection'], index = ['Province/State','Country/Region']),ignore_index = True)
    
    df = df.groupby('Country/Region').sum()
    df = df.T
    df.columns.name = 'Country'
    df.index.name = 'Date'
    
    if len(arg) == 0:
        arg = df.columns
    df = df[arg]
        
    for i in arg:
        country[i+'_Daily'] = country[[i]].diff(axis = 0, periods = 1)
    country.columns.name = 'Country'
    country.index.name = 'Date'
    return country

def plot():
    col1 = sorted([i for i in sick.columns if 'Daily' in i])
    col2 = sorted([i for i in sick.columns if 'Daily' not in i])
    
    sick[col2].plot(kind ='bar', figsize = (15,3), title = 'Total reported cases');
    sick[col1].plot(kind ='bar', figsize = (15,3), title = 'Daily increase on cases');
    dead[col2].plot(kind ='bar', figsize = (15,3), title = 'Sum of death');
    dead[col1].plot(kind ='bar', figsize = (15,3), title = 'Daily increase on deaths');
    

In [6]:
s0 = 'time_series_covid19_confirmed_global.csv'
s1 = 'time_series_covid19_deaths_global.csv'
arg = []
sick = infect(s0,*arg)

In [61]:
#notebook version dropdown menu
from bokeh.models import ColumnDataSource
import warnings
warnings.filterwarnings('ignore')
from bokeh.models import ColumnarDataSource, CDSView, GroupFilter, WidgetBox, CheckboxGroup
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.layouts import row, WidgetBox, column
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, Select
from bokeh.embed import file_html

s0 = 'time_series_covid19_confirmed_global.csv'
s1 = 'time_series_covid19_deaths_global.csv'
arg = []
sick = infect(s0,*arg)
dead = infect(s1,*arg)

def modify(doc):
    def make_data(selection_sum):

        data_cds = pd.DataFrame()
        #chkbox_active = chkbox_active1 #+ [dropdown.value]
        s_cds = sick[selection_sum]
        s_cds.reset_index(inplace= True)
        s_cds['Date'] = pd.to_datetime(s_cds['Date'])
        s_cds = pd.melt(s_cds, id_vars= ['Date'], value_vars=s_cds.columns.to_list()[1:])
        return ColumnDataSource(s_cds)

    def make_plot(src):
            '''
            makes plots according to the view list


            hover_display = {'Date': '@gmDate{(%Y-%m-%d)}'}
            hover_display = """
            <div>Display</div
            <div>Team: @teamAbbr</div>
            <div><span>Win</span><span style='padding-left:40px; color: red'>@winLoss</span></div>

            '''               
            colour_map = colour_map = CategoricalColorMapper(factors = src.data['Country'], palette = ['red', 'green','orange'])
            fig_kwargs = {'title' : 'Sick            '
                          ,'plot_width': 800
                          ,'plot_height': 200
                           ,'x_axis_type': 'datetime'
                          }
            plot_kwargs = {'source': src
                           ,'width': 0.9
                           ,'color': {'field': 'Country', 'transform': colour_map}
                          }

            fig = figure(**fig_kwargs)
            fig.vbar(x='Date',top='value', **plot_kwargs)
            #fig.add_tools(HoverTool(tooltips= hover_display))
            

            return fig
    checkbox_group1 = CheckboxGroup(
        labels=['China','United Kingdom', 'US', 'Italy'], active=[0]) #list of checkbox limited
    
    dropdown1 = Select(title="Select country",value='China',options=sick.columns.to_list())
    dropdown2 = Select(title="Select country",value='China',options=sick.columns.to_list())
    
    def update(attr, old, new):
        
        
        chkbox_actives1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
        selection_sum = chkbox_actives1 +[dropdown1.value]+[dropdown2.value]
        new_src = make_data(selection_sum)
                  
        src.data.update(new_src.data)
        return src
          

    checkbox_group1.on_change('active', update)
    dropdown1.on_change('value', update)
    dropdown2.on_change('value', update)
    
    controls1 = WidgetBox(checkbox_group1, sizing_mode = 'fixed', height= 60, width = 130) 
    controls2 = WidgetBox(dropdown1, sizing_mode = 'fixed', height= 50, width = 100)
    controls3 = WidgetBox(dropdown2, sizing_mode = 'fixed', height= 50, width = 100)

    
    start_selection1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
    start_selection_sum = start_selection1 + [dropdown1.value] + [dropdown2.value]
    
    src = make_data(start_selection_sum)

    fig = make_plot(src)
    
    layout = column(row(controls1,controls2, controls3),fig)
    tab = Panel(child=layout, title = 'wins')
    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)
    return doc


# Set up an application
output_notebook()
handler = FunctionHandler(modify)
app = Application(handler)

#resources = ['https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.js','https://cdn.bokeh.org/bokeh/release/bokeh-widgets-x.y.z.min.js']
#file_html(app, resources=resources)
show(app)


Loading BokehJS ...

In [98]:
df = pd.read_csv(s0)
df.drop(['Lat','Long'], axis = 1, inplace=True)
df = df.append( pd.Series([np.NAN, 'World'], index = ['Province/State','Country/Region']).append(df.sum()), ignore_index =True)
df = df.append(pd.Series([np.NAN, 'No selection'], index = ['Province/State','Country/Region']),ignore_index = True)
df = df.groupby('Country/Region').sum()
df = df.T
df.columns.name = 'Country'
df.index.name = 'Date'
df = df[['China','US']]

In [3]:
#notebook version multicheckbox
from bokeh.models import ColumnDataSource
import warnings
warnings.filterwarnings('ignore')
from bokeh.models import ColumnarDataSource, GroupFilter, WidgetBox, CheckboxGroup
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.layouts import row, WidgetBox, column
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs

s0 = 'time_series_covid19_confirmed_global.csv'
s1 = 'time_series_covid19_deaths_global.csv'
arg = []
sick = infect(s0,*arg)
dead = infect(s1,*arg)

def modify(doc):
    def make_data(chkbox_active1,chkbox_active2,chkbox_active3,chkbox_active4,chkbox_active5):

        data_cds = pd.DataFrame()
        chkbox_active = chkbox_active1 + chkbox_active2+ chkbox_active3+ chkbox_active4+ chkbox_active5
        s_cds = sick[chkbox_active]
        s_cds.reset_index(inplace= True)
        s_cds['Date'] = pd.to_datetime(s_cds['Date'])
        s_cds = pd.melt(s_cds, id_vars= ['Date'], value_vars=s_cds.columns.to_list()[1:])
        return ColumnDataSource(s_cds)

    def make_plot(src):
            '''
            makes plots according to the view list


            hover_display = {'Date': '@gmDate{(%Y-%m-%d)}'}
            hover_display = """
            <div>Display</div
            <div>Team: @teamAbbr</div>
            <div><span>Win</span><span style='padding-left:40px; color: red'>@winLoss</span></div>

            '''               

            fig_kwargs = {'title' : 'Sick            '
                          ,'plot_width': 400
                          ,'plot_height': 200
                           ,'x_axis_type': 'datetime'
                          }
            plot_kwargs = {'source': src
                          }

            fig = figure(**fig_kwargs)
            fig.triangle(x='Date',y='value', **plot_kwargs)
            #fig.add_tools(HoverTool(tooltips= hover_display))

            return fig
    checkbox_group1 = CheckboxGroup(
        labels=sick.columns.tolist()[:15], active=[0]) #list of checkbox limited
    checkbox_group2 = CheckboxGroup(
        labels=sick.columns.tolist()[15:30], active=[1]) #list of checkbox limited
    checkbox_group3 = CheckboxGroup(
        labels=sick.columns.tolist()[30:45], active=[0])
    checkbox_group4 = CheckboxGroup(
        labels=sick.columns.tolist()[45:60], active=[0])
    checkbox_group5 = CheckboxGroup(
        labels=sick.columns.tolist()[60:75], active=[0])
    def update(attr, old, new):
        
        
        chkbox_actives1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
        chkbox_actives2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
        chkbox_actives3 = [checkbox_group3.labels[i] for i in checkbox_group3.active]
        chkbox_actives4 = [checkbox_group4.labels[i] for i in checkbox_group4.active]
        chkbox_actives5 = [checkbox_group5.labels[i] for i in checkbox_group5.active]
        new_src = make_data(chkbox_actives1,chkbox_actives2,chkbox_actives3,chkbox_actives4,chkbox_actives5)
                  
        src.data.update(new_src.data)
        return src
          

    checkbox_group1.on_change('active', update)
    checkbox_group2.on_change('active', update)
    checkbox_group3.on_change('active', update)
    checkbox_group4.on_change('active', update)
    checkbox_group5.on_change('active', update)
    controls1 = WidgetBox(checkbox_group1, sizing_mode = 'fixed', height= 250, width = 80) 
    controls2 = WidgetBox(checkbox_group2, sizing_mode = 'fixed', height= 250, width = 80)
    controls3 = WidgetBox(checkbox_group3, sizing_mode = 'fixed', height= 250, width = 80)
    controls4 = WidgetBox(checkbox_group4, sizing_mode = 'fixed', height= 250, width = 80)
    controls5 = WidgetBox(checkbox_group5, sizing_mode = 'fixed', height= 250, width = 80)

    
    start_selection1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
    start_selection2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
    start_selection3 = [checkbox_group3.labels[i] for i in checkbox_group3.active]
    start_selection4 = [checkbox_group4.labels[i] for i in checkbox_group4.active]
    start_selection5 = [checkbox_group5.labels[i] for i in checkbox_group5.active]
    src = make_data(start_selection1,start_selection2,start_selection3,start_selection4,start_selection5)

    fig = make_plot(src)
    
    layout = row(controls1,controls2, fig, controls3, controls4, controls5)
    tab = Panel(child=layout, title = 'wins')
    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)
    return doc


# Set up an application
output_notebook()
handler = FunctionHandler(modify)
app = Application(handler)

#resources = ['https://cdn.bokeh.org/bokeh/release/bokeh-1.3.4.min.js','https://cdn.bokeh.org/bokeh/release/bokeh-widgets-x.y.z.min.js']
#file_html(app, resources=resources)
show(app)


Loading BokehJS ...

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8889';                       use --allow-websocket-origin=localhost:8889 or set BOKEH_ALLOW_WS_ORIGIN=localhost:8889 to permit this; currently we allow origins {'localhost:8888'}


In [12]:
# server version multicheckbox
from bokeh.models import ColumnDataSource
import warnings
warnings.filterwarnings('ignore')
from bokeh.models import ColumnarDataSource, CDSView, GroupFilter, WidgetBox, CheckboxGroup
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.layouts import row, WidgetBox, column
from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure
from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs


def make_data(chkbox_active1,chkbox_active2,chkbox_active3,chkbox_active4,chkbox_active5):

    data_cds = pd.DataFrame()
    chkbox_active = chkbox_active1 + chkbox_active2+ chkbox_active3+ chkbox_active4+ chkbox_active5
    s_cds = sick[chkbox_active]
    s_cds.reset_index(inplace= True)
    s_cds['Date'] = pd.to_datetime(s_cds['Date'])
    s_cds = pd.melt(s_cds, id_vars= ['Date'], value_vars=s_cds.columns.to_list()[1:])
    return ColumnDataSource(s_cds)

def make_plot(src):

    fig_kwargs = {'title' : 'Sick            '
                  ,'plot_width': 400
                  ,'plot_height': 200
                  ,'x_axis_type': 'datetime'
                  }
    plot_kwargs = {'source': src
      }

    fig = figure(**fig_kwargs)
    fig.triangle(x='Date',y='value', **plot_kwargs)
    #fig.add_tools(HoverTool(tooltips= hover_display))

    return fig

def update(attr, old, new):
  
  
    chkbox_actives1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
    chkbox_actives2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
    chkbox_actives3 = [checkbox_group3.labels[i] for i in checkbox_group3.active]
    chkbox_actives4 = [checkbox_group4.labels[i] for i in checkbox_group4.active]
    chkbox_actives5 = [checkbox_group5.labels[i] for i in checkbox_group5.active]
    new_src = make_data(chkbox_actives1,chkbox_actives2,chkbox_actives3,chkbox_actives4,chkbox_actives5)

    src.data.update(new_src.data)
    return src
 
checkbox_group1 = CheckboxGroup(
  labels=sick.columns.tolist()[:15], active=[0]) #list of checkbox limited
checkbox_group2 = CheckboxGroup(
  labels=sick.columns.tolist()[15:30], active=[1]) #list of checkbox limited
checkbox_group3 = CheckboxGroup(
  labels=sick.columns.tolist()[30:45], active=[0])
checkbox_group4 = CheckboxGroup(
  labels=sick.columns.tolist()[45:60], active=[0])
checkbox_group5 = CheckboxGroup(
  labels=sick.columns.tolist()[60:75], active=[0])

checkbox_group1.on_change('active', update)
checkbox_group2.on_change('active', update)
checkbox_group3.on_change('active', update)
checkbox_group4.on_change('active', update)
checkbox_group5.on_change('active', update)
controls1 = WidgetBox(checkbox_group1, sizing_mode = 'fixed', height= 250, width = 80) 
controls2 = WidgetBox(checkbox_group2, sizing_mode = 'fixed', height= 250, width = 80)
controls3 = WidgetBox(checkbox_group3, sizing_mode = 'fixed', height= 250, width = 80)
controls4 = WidgetBox(checkbox_group4, sizing_mode = 'fixed', height= 250, width = 80)
controls5 = WidgetBox(checkbox_group5, sizing_mode = 'fixed', height= 250, width = 80)

 
start_selection1 = [checkbox_group1.labels[i] for i in checkbox_group1.active]
start_selection2 = [checkbox_group2.labels[i] for i in checkbox_group2.active]
start_selection3 = [checkbox_group3.labels[i] for i in checkbox_group3.active]
start_selection4 = [checkbox_group4.labels[i] for i in checkbox_group4.active]
start_selection5 = [checkbox_group5.labels[i] for i in checkbox_group5.active]
src = make_data(start_selection1,start_selection2,start_selection3,start_selection4,start_selection5)

fig = make_plot(src)
 
layout = row(controls1,controls2, fig, controls3, controls4, controls5)
tab = Panel(child=layout, title = 'wins')
tabs = Tabs(tabs=[tab])


# Set up an application
output_notebook()
curdoc().add_root(tabs)

Loading BokehJS ...